<a href="https://colab.research.google.com/github/soerenetler/spacy_target_answer/blob/master/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!(cd spacy_target_answer; git pull)
!(cd squad-question-generation/; git pull)

Already up to date.
Already up to date.


In [3]:
!cp crf_target_answer/Custom.py Custom.py

In [4]:
!nvidia-smi

Sun Jun 13 14:15:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
#!pip install --quiet -U pip setuptools wheel

In [6]:
!pip install --quiet -U spacy[cuda110,transformers,lookups]

     |████████████████████████████████| 12.8MB 8.1MB/s 
     |████████████████████████████████| 624kB 31.9MB/s 
     |████████████████████████████████| 9.1MB 30.8MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 460kB 35.7MB/s 
     |████████████████████████████████| 165.3MB 82kB/s 
     |████████████████████████████████| 97.3MB 118kB/s 
     |████████████████████████████████| 122kB 39.4MB/s 
     |████████████████████████████████| 2.1MB 31.9MB/s 
     |████████████████████████████████| 1.0MB 24.8MB/s 
     |████████████████████████████████| 901kB 24.9MB/s 
     |████████████████████████████████| 3.3MB 29.8MB/s 


In [7]:
!python -m spacy download en_core_web_sm --quiet
!python -m spacy download en_core_web_trf --quiet
!python -m spacy download en_core_web_lg --quiet

2021-06-13 14:16:43.931539: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 8.1MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2021-06-13 14:16:52.915012: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7MB 37kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
2021-06-13 14:17:34.375045: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8MB 25kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [8]:
!spacy validate

2021-06-13 14:18:41.797272: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.0.6) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_trf   >=3.0.0,<3.1.0   3.0.0   ✔
en_core_web_lg    >=3.0.0,<3.1.0   3.0.0   ✔



In [9]:
# Train NER
!python -m spacy project run corpus spacy_target_answer

2021-06-13 14:18:45.603758: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

=================================== corpus ===================================
Running command: /usr/bin/python3 -m spacy convert /content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/quac/sentences_conll corpus -c ner
2021-06-13 14:18:50.906605: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (2302 documents): corpus/dev.spacy
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (1000 documents): corpus/test.spacy
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
✔ Generated output file (9263 documents): corpus/train.spacy


In [10]:
!python -m spacy init fill-config ./spacy_target_answer/configs/base_config.cfg ./spacy_target_answer/configs/config.cfg
!python -m spacy project run train spacy_target_answer

2021-06-13 14:20:24.415376: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
spacy_target_answer/configs/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
2021-06-13 14:20:29.141087: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

=================================== train ===================================
Running command: /usr/bin/python3 -m spacy train configs/config.cfg -o training/ --gpu-id 0 --paths.train corpus/train.spacy --paths.dev corpus/dev.spacy --output /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training
2021-06-13 14:20:32.469088: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

In [11]:
!python -m spacy project run evaluate spacy_target_answer

2021-06-13 20:43:46.329250: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

================================== evaluate ==================================
Running command: /usr/bin/python3 -m spacy evaluate /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best ./corpus/test.spacy --output /content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/metrics/config.json --gpu-id 0
2021-06-13 20:43:52.525474: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     -    
NER P   43.62
NER R   10.37
NER F   16.76
SPEED   5773 


=============================== NER (per type) ===============================

             P       R       F
Answer   43.62   10.37   16.76

✔ Saved results to
/content/gdrive/MyDrive/mt-

In [12]:
import spacy
nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/quac/training/model-best')
doc = nlp(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin. [...]")
spacy.displacy.render(doc, style='ent', jupyter =True)

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [13]:
nlp2 = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/squad/training/model-best')
doc = nlp2(u"There are no major roads through Golm, only state road (Landesstraße) L 902 passes through the northern parts of the village area and offers the shortest connection to federal highway (Bundesstraße) 273.  The latter can also be reached following a road extending east from Golm past Sanssouci Park to the Jägervorstadt quarter of Potsdam. The railway between Wildpark and Nauen was opened in 1902 as part of the single-tracked bypass railway (Umgehungsbahn) intended to connect the radial railway lines originating from Berlin.  [...]")
spacy.displacy.render(doc, style='ent', jupyter =True)

In [14]:
pip install scikit-learn sklearn_crfsuite -U

     |████████████████████████████████| 22.3MB 1.6MB/s 
     |████████████████████████████████| 747kB 29.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [20]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from tqdm import tqdm
import ast
from Custom import *
import re

spacy.require_gpu()
dataset = "quac"
df = pd.read_csv("/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"+dataset+"/paragraph/test.csv")
df["askable_tokens"] = [ast.literal_eval(t) for t in df["askable_tokens"]]
df["paragraph_tokens"] = [ast.literal_eval(t) for t in df["paragraph_tokens"]]

print(df.columns)

nlp = spacy.load('/content/gdrive/MyDrive/mt-qg-data/00_models/spacy_target_answer/'+dataset+'/training/model-best')
def custom_tokenizer(nlp):
    return lambda text : Doc(nlp.vocab, words=text)

nlp.tokenizer = custom_tokenizer(nlp)
nlp.add_pipe("sentencizer")


pred_y = []
true_y = []

with open("./trf_results.txt", "a") as file:
  for doc, y in tqdm(zip(nlp.pipe(df["paragraph_tokens"]), io2iob(df["askable_tokens"]))):
    labels = []
    for sent in doc.sents:
      for token, y_ in zip(sent, y[sent.start:sent.end]):
        if token.ent_iob_ =="O":
          labels.append("O")
          label_ = "O"
        else:
          labels.append(token.ent_iob_ + "-" + token.ent_type_)
          label_ = token.ent_iob_ + "-" + token.ent_type_
        file.write("{} {} {}\n".format(token.text, y_, label_))
      pred_y.append(labels)
      true_y.append(y) 
      file.write("\n")

Index(['Unnamed: 0', 'text_title', 'paragraph_text', 'paragraph_tokens',
       'askable_tokens'],
      dtype='object')


1000it [09:26,  1.77it/s]


In [21]:
from sklearn_crfsuite import metrics
metrics.flat_classification_report(io2iob(true_y), pred_y, digits=5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


'              precision    recall  f1-score   support\n\n    B-Answer    0.35541   0.15496   0.21582    135815\n    I-Answer    0.48069   0.15909   0.23907   4177401\n           O    0.70022   0.91918   0.79490   9182586\n\n    accuracy                        0.67622  13495802\n   macro avg    0.51211   0.41108   0.41660  13495802\nweighted avg    0.62880   0.67622   0.61702  13495802\n'

In [22]:
pairs = {
  "answer_sentence_token": [],
  "answer": [],
  "answer_token": [],
  "question_token": [],
}

for doc in tqdm(nlp.pipe(df["paragraph_tokens"])):
  for sent in doc.sents:
    for entity in sent.ents:
      pairs["answer_sentence_token"].append([t.text for t in sent])
      pairs["answer"].append(entity.text)
      pairs["answer_token"].append([t.text for t in entity])
      pairs["question_token"].append([])

1000it [01:23, 11.99it/s]


In [23]:
target_answer_seq = pd.DataFrame(pairs)

In [24]:
target_answer_seq.to_csv("/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"+dataset+"/question_answer/trf_test.csv")